In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df1 = pd.read_excel("/content/drive/MyDrive/Tacos_data/Telugu_Tacos_Train.xlsx")

In [4]:
df = df1

In [5]:
#df2 = pd.read_excel("/content/Mizo_Tacos_Train.xlsx")

In [6]:
#df =  pd.concat([df1, df2])

In [7]:
#converting the labels to numbers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['labels'] = le.fit_transform(df.Sentiment.values)

In [8]:
from sklearn.utils import shuffle
df = shuffle(df)

In [9]:
positive_labels = (df['labels'] == 1).sum()
negative_labels = (df['labels'] == 0).sum()

In [10]:
print(f"Number of positive labels: {positive_labels}")
print(f"Number of negative labels: {negative_labels}")

Number of positive labels: 400
Number of negative labels: 400


In [11]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels_str, val_labels_str = train_test_split(list(df['Text']), list(df['Sentiment']), test_size=.2)

In [12]:
!pip install sentencepiece

In [13]:
!pip install transformers

In [14]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [15]:
!pip install torch==1.11.00.676617

ERROR: Could not find a version that satisfies the requirement torch==1.11.00.676617 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0)
ERROR: No matching distribution found for torch==1.11.00.676617


In [16]:
import torch

In [17]:
from transformers import EarlyStoppingCallback, IntervalStrategy,Trainer, TrainingArguments

In [18]:
!pip install --upgrade accelerate

In [19]:
!pip install accelerate -U

In [20]:
!pip install transformers[torch]

In [21]:
!pip install accelerate>=0.21.0

In [22]:
!pip install --upgrade accelerate

In [23]:
import torch
torch.cuda.empty_cache()

In [24]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [25]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", do_lower_case=False, use_fast=False, keep_accents=True)
model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=2)

# Tokenize input texts
train_encodings = tokenizer(train_texts, padding=True, truncation=True,max_length=512)
val_encodings = tokenizer(val_texts, padding=True, truncation=True,max_length=512)

# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert string labels to integers
train_labels = le.fit_transform(train_labels_str)
val_labels = le.transform(val_labels_str)

# Create dataset objects
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Define a function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc, 'F1': f1, 'Precision': precision, 'Recall': recall}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=15,
    per_device_eval_batch_size=15,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10
)

# Create a trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.690200,0.709482,0.387500,0.387500,0.387500,0.387500
20,0.689800,0.709705,0.387500,0.387500,0.387500,0.387500
30,0.695400,0.709301,0.387500,0.387500,0.387500,0.387500
40,0.689600,0.709092,0.387500,0.387500,0.387500,0.387500
50,0.683200,0.709178,0.387500,0.387500,0.387500,0.387500
60,0.688500,0.710867,0.387500,0.387500,0.387500,0.387500
70,0.691500,0.711473,0.387500,0.387500,0.387500,0.387500
80,0.699900,0.709596,0.387500,0.387500,0.387500,0.387500
90,0.696800,0.707101,0.387500,0.387500,0.387500,0.387500
100,0.688400,0.705743,0.387500,0.387500,0.387500,0.387500


TrainOutput(global_step=215, training_loss=0.6838934754216394, metrics={'train_runtime': 374.2329, 'train_samples_per_second': 8.551, 'train_steps_per_second': 0.575, 'total_flos': 76473925632000.0, 'train_loss': 0.6838934754216394, 'epoch': 5.0})

In [26]:
metrics=trainer.evaluate()

In [27]:
metrics

{'eval_loss': 0.596554160118103,
 'eval_Accuracy': 0.5875,
 'eval_F1': 0.5875,
 'eval_Precision': 0.5875,
 'eval_Recall': 0.5875,
 'eval_runtime': 5.3784,
 'eval_samples_per_second': 29.749,
 'eval_steps_per_second': 2.045,
 'epoch': 5.0}

In [28]:
test_df = pd.read_excel("/content/drive/MyDrive/Tacos_data/mizo_data_tacos_test_final(1).xlsx")

In [29]:
len(test_df)

200

In [30]:
test_df.head()

,Text,Sentiment
0,"He hla hi hmangaihna lamah te, min kawngah te ...",POSITIVE
1,He track-a guitar hnathawh inzawmkhawm leh sou...,POSITIVE
2,Genre hrang hrang inzawmkhawmna famkim ani . H...,POSITIVE
3,"A hla thu hi hla thu a ni a, a thluk pawh a ma...",POSITIVE
4,Positivity pholang thei hla tichaktu. Artist-i...,POSITIVE


In [31]:
test_texts = list(test_df['Text'])

In [32]:
test_labels = le.transform(test_df['Sentiment'])

In [33]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = CustomDataset(test_encodings, test_labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
# here we are evaluating a model using the Trainer object trainer on a test dataset.
test_metrics = trainer.evaluate(test_dataset)

In [35]:
test_metrics

{'eval_loss': 0.6905487179756165,
 'eval_Accuracy': 0.56,
 'eval_F1': 0.56,
 'eval_Precision': 0.56,
 'eval_Recall': 0.56,
 'eval_runtime': 7.0734,
 'eval_samples_per_second': 28.275,
 'eval_steps_per_second': 1.979,
 'epoch': 5.0}